## Exercise: Select Optimal Model by Tuning Hyperparameters

Use grid search and cross-validation to tune the hyperparameters from a logistic regression model.

Run the following cell to set up our environment.

In [0]:
%run "./Includes/Classroom-Setup"

### Step 1: Import the Data

Import the data and perform a train/test split.

In [0]:
from pyspark.sql.functions import col

cols = ["index",
 "sample-code-number",
 "clump-thickness",
 "uniformity-of-cell-size",
 "uniformity-of-cell-shape",
 "marginal-adhesion",
 "single-epithelial-cell-size",
 "bare-nuclei",
 "bland-chromatin",
 "normal-nucleoli",
 "mitoses",
 "class"]

cancerDF = (spark.read  # read the data
  .option("HEADER", True)
  .option("inferSchema", True)
  .csv("/mnt/training/cancer/biopsy/biopsy.csv")
)

cancerDF = (cancerDF    # Add column names and drop nulls
  .toDF(*cols)
  .withColumn("bare-nuclei", col("bare-nuclei").isNotNull().cast("integer"))
)

display(cancerDF)

index,sample-code-number,clump-thickness,uniformity-of-cell-size,uniformity-of-cell-shape,marginal-adhesion,single-epithelial-cell-size,bare-nuclei,bland-chromatin,normal-nucleoli,mitoses,class
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,1,3,2,1,benign
3,1015425,3,1,1,1,2,1,3,1,1,benign
4,1016277,6,8,8,1,3,1,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,1,9,7,1,malignant
7,1018099,1,1,1,1,2,1,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign
10,1033078,4,2,1,1,2,1,2,1,1,benign


In [0]:
cancerDF.where('`bare-nuclei` IS NULL').count() # find col by col

Out[8]: 0

Perform a train/test split to create `trainCancerDF` and `testCancerDF`.  Put 80% of the data in `trainCancerDF` and use the seed that is set for you.

In [0]:
# TODO
seed = 42
trainCancerDF, testCancerDF = cancerDF.randomSplit([.8, .2], seed=seed)

### Step 2: Create a Pipeline

Create a pipeline `cancerPipeline` that consists of the following stages:<br>

1. `indexer`: a `StringIndexer` that takes `class` as an input and outputs the column `is-malignant`
2. `assembler`: a `VectorAssembler` that takes all of the other columns as an input and outputs  the column `features`
3. `logr`: a `LogisticRegression` that takes `features` as the input and `is-malignant` as the output variable

In [0]:
# TODO
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

indexer = StringIndexer(inputCol='class', outputCol='is-malignant')
assembler = VectorAssembler(inputCols=cols[2:-1], outputCol='features')
logr = LogisticRegression(featuresCol='features', labelCol='is-malignant')
cancerPipeline = Pipeline(stages=[indexer, assembler, logr])

In [0]:
# TEST - Run this cell to test your solution
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

assert type(indexer) == type(StringIndexer())
assert indexer.getInputCol() == 'class'
assert indexer.getOutputCol() == 'is-malignant'

assert type(assembler) == type(VectorAssembler())
assert assembler.getInputCols() == cols[2:-1]
assert assembler.getOutputCol() == 'features'

assert type(logr) == type(LogisticRegression())
assert logr.getLabelCol() == "is-malignant"
assert logr.getFeaturesCol() == 'features'

assert type(cancerPipeline) == type(Pipeline())

# dbTest("ML1-P-08-02-01", True, type(indexer) == type(StringIndexer()))
# dbTest("ML1-P-08-02-02", True, indexer.getInputCol() == 'class')
# dbTest("ML1-P-08-02-03", True, indexer.getOutputCol() == 'is-malignant')

# dbTest("ML1-P-08-02-04", True, type(assembler) == type(VectorAssembler()))
# dbTest("ML1-P-08-02-05", True, assembler.getInputCols() == cols[2:-1])
# dbTest("ML1-P-08-02-06", True, assembler.getOutputCol() == 'features')

# dbTest("ML1-P-08-02-07", True, type(logr) == type(LogisticRegression()))
# dbTest("ML1-P-08-02-08", True, logr.getLabelCol() == "is-malignant")
# dbTest("ML1-P-08-02-09", True, logr.getFeaturesCol() == 'features')

# dbTest("ML1-P-08-02-10", True, type(cancerPipeline) == type(Pipeline()))

print("Tests passed!")

Tests passed!

### Step 3: Create Grid Search Parameters

Take a look at the parameters for our `LogisticRegression` object.  Use this to build the inputs to grid search.

In [0]:
print(logr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: is-malignant)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

Create a `ParamGridBuilder` object with two grids:<br><br>

1. A regularization parameter `regParam` of `[0., .2, .8, 1.]`
2. Test both with and without an intercept using `fitIntercept`

In [0]:
# TODO
from pyspark.ml.tuning import ParamGridBuilder

cancerParamGrid = (
  ParamGridBuilder()
  .addGrid(logr.regParam, [0., .2, .8, 1.])
  .addGrid(logr.fitIntercept, [True, False])
  .build()
 )

In [0]:
# TEST - Run this cell to test your solution
# dbTest("ML1-P-08-03-01", True, type(cancerParamGrid) == list)
assert type(cancerParamGrid) == list
print("Tests passed!")

Tests passed!

### Step 4: Perform 3-Fold Cross-Validation

Create a `BinaryClassificationEvaluator` object and use it to perform 3-fold cross-validation.

In [0]:
# TODO
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator

binaryEvaluator = BinaryClassificationEvaluator(
  labelCol = "is-malignant", 
  metricName = "areaUnderROC"
)

cancerCV = CrossValidator(
  estimator = cancerPipeline,              # Estimator (individual model or pipeline)
  estimatorParamMaps = cancerParamGrid ,    # Grid of parameters to try (grid search)
  evaluator= binaryEvaluator,               # Evaluator
  numFolds = 3,               # Set k to 3
  seed = 42                           # Seed to sure our results are the same if ran again
)

cancerCVModel = cancerCV.fit(trainCancerDF)

In [0]:
# TEST - Run this cell to test your solution
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator

# dbTest("ML1-P-08-04-01", True, type(binaryEvaluator) == type(BinaryClassificationEvaluator()))
# dbTest("ML1-P-08-04-02", True, type(cancerCV) == type(CrossValidator()))

assert type(binaryEvaluator) == type(BinaryClassificationEvaluator())
assert type(cancerCV) == type(CrossValidator())

print("Tests passed!")

Tests passed!

### Step 5: Examine the results

Take a look at the results.  Which combination of hyperparameters learned the most from the data?

In [0]:
for params, score in zip(cancerCVModel.getEstimatorParamMaps(), cancerCVModel.avgMetrics):
  print("".join([param.name+"\t"+str(params[param])+"\t" for param in params]))
  print("\tScore: {}".format(score))

regParam	0.0	fitIntercept	True	
	Score: 0.9902258029062572
regParam	0.0	fitIntercept	False	
	Score: 0.8321216911039537
regParam	0.2	fitIntercept	True	
	Score: 0.9896529273437495
regParam	0.2	fitIntercept	False	
	Score: 0.8830757170204708
regParam	0.8	fitIntercept	True	
	Score: 0.9891246547731314
regParam	0.8	fitIntercept	False	
	Score: 0.9564332230874579
regParam	1.0	fitIntercept	True	
	Score: 0.9890554699506727
regParam	1.0	fitIntercept	False	
	Score: 0.9607487160319952